# 🎓 Face Recognition Training Notebook

Notebook này sẽ hướng dẫn bạn train lại Face Recognition model từ dataset có sẵn.

## 📋 Các bước thực hiện:
1. Kiểm tra dataset
2. Load và preprocess dữ liệu
3. Train Face Recognition model
4. Test và đánh giá model
5. Lưu model đã train


## 📦 Import các thư viện cần thiết


In [2]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import pickle
from pathlib import Path

# Computer Vision
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Machine Learning
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# Face Processing
from mtcnn.mtcnn import MTCNN

from keras_facenet import FaceNet

# Data Augmentation
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from numpy import savez_compressed, asarray, load, expand_dims

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


## 📁 Kiểm tra Dataset Structure


In [3]:
# Đường dẫn dataset
dataset_train = "FaceDataset/train/"
dataset_val = "FaceDataset/val/"

print("🔍 Checking dataset structure...")
print("=" * 50)

# Kiểm tra thư mục train
if os.path.exists(dataset_train):
    print(f"✅ Training dataset found: {dataset_train}")
    
    # Đếm số người và ảnh
    people_count = 0
    total_images = 0
    
    for person_name in os.listdir(dataset_train):
        person_dir = os.path.join(dataset_train, person_name)
        if os.path.isdir(person_dir):
            people_count += 1
            images = [f for f in os.listdir(person_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            image_count = len(images)
            total_images += image_count
            print(f"   👤 {person_name}: {image_count} images")
    
    print(f"\n📊 Training Summary:")
    print(f"   👥 Total people: {people_count}")
    print(f"   🖼️  Total images: {total_images}")
else:
    print(f"❌ Training dataset not found: {dataset_train}")

print("\n" + "=" * 50)

# Kiểm tra thư mục validation
if os.path.exists(dataset_val):
    print(f"✅ Validation dataset found: {dataset_val}")
    
    val_people_count = 0
    val_total_images = 0
    
    for person_name in os.listdir(dataset_val):
        person_dir = os.path.join(dataset_val, person_name)
        if os.path.isdir(person_dir):
            val_people_count += 1
            images = [f for f in os.listdir(person_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            image_count = len(images)
            val_total_images += image_count
            print(f"   👤 {person_name}: {image_count} images")
    
    print(f"\n📊 Validation Summary:")
    print(f"   👥 Total people: {val_people_count}")
    print(f"   🖼️  Total images: {val_total_images}")
else:
    print(f"❌ Validation dataset not found: {dataset_val}")


🔍 Checking dataset structure...
✅ Training dataset found: FaceDataset/train/
   👤 ben_afflek: 14 images
   👤 elton_john: 17 images
   👤 mindy_kaling: 22 images
   👤 messi: 15 images
   👤 jerry_seinfeld: 21 images

📊 Training Summary:
   👥 Total people: 5
   🖼️  Total images: 89

✅ Validation dataset found: FaceDataset/val/
   👤 ben_afflek: 5 images
   👤 elton_john: 5 images
   👤 mindy_kaling: 5 images
   👤 messi: 5 images
   👤 jerry_seinfeld: 5 images

📊 Validation Summary:
   👥 Total people: 5
   🖼️  Total images: 25


## 🔧 Khởi tạo Face Processing Models


In [4]:
print("🔧 Initializing face processing models...")

# Khởi tạo MTCNN cho face detection
print("📷 Loading MTCNN face detector...")
mtcnn = MTCNN()
print("✅ MTCNN loaded successfully")

# Khởi tạo FaceNet cho face embeddings
print("🧠 Loading FaceNet model...")
facenet = FaceNet()
print("✅ FaceNet loaded successfully")

print("\n🎉 All models initialized successfully!")


🔧 Initializing face processing models...
📷 Loading MTCNN face detector...
✅ MTCNN loaded successfully
🧠 Loading FaceNet model...
✅ FaceNet loaded successfully

🎉 All models initialized successfully!


## 🖼️ Hàm xử lý ảnh và trích xuất khuôn mặt


In [5]:
def extract_face(image_path, required_size=(160, 160)):
    """Trích xuất khuôn mặt từ ảnh sử dụng MTCNN"""
    try:
        # Load ảnh
        image = Image.open(image_path)
        image = image.convert('RGB')
        pixels = np.asarray(image)
        
        # Detect faces
        results = mtcnn.detect_faces(pixels)
        
        if results:
            # Lấy khuôn mặt đầu tiên
            x1, y1, width, height = results[0]['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            
            # Extract face
            face = pixels[y1:y2, x1:x2]
            
            # Resize
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array = np.asarray(image)
            
            return face_array
        
        return None
    
    except Exception as e:
        print(f"⚠️  Error processing {image_path}: {e}")
        return None

def get_face_embedding(face_pixels):
    """Lấy face embedding từ FaceNet"""
    try:
        # Transform face into one sample
        samples = expand_dims(face_pixels, axis=0)
        
        # Get embedding
        yhat = facenet.embeddings(samples)
        return yhat[0]
    
    except Exception as e:
        print(f"⚠️  Error getting embedding: {e}")
        return None

print("✅ Face processing functions defined")


✅ Face processing functions defined


## 📊 Load và Preprocess Dataset


In [6]:
def load_dataset(directory, use_augmentation=True):
    """Load dataset từ thư mục với data augmentation"""
    X = []
    y = []
    
    # Data augmentation generator
    if use_augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=True,
            featurewise_std_normalization=True,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True
        )
    
    print(f"🔄 Loading dataset from: {directory}")
    
    for person_name in os.listdir(directory):
        person_dir = os.path.join(directory, person_name)
        
        if not os.path.isdir(person_dir):
            continue
        
        print(f"\n👤 Processing {person_name}...")
        
        # Load tất cả ảnh của người này
        faces = []
        for filename in os.listdir(person_dir):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(person_dir, filename)
                face = extract_face(image_path)
                
                if face is not None:
                    faces.append(face)
        
        print(f"   📸 Found {len(faces)} valid faces")
        
        # Thêm faces và labels
        for face in faces:
            X.append(face)
            y.append(person_name)
            
            # Data augmentation
            if use_augmentation:
                augmented_count = 0
                for x in datagen.flow(expand_dims(face, axis=0), batch_size=1):
                    X.append(x[0])
                    y.append(person_name)
                    augmented_count += 1
                    if augmented_count >= 3:  # Tạo thêm 3 ảnh từ mỗi ảnh gốc
                        break
    
    print(f"\n📊 Dataset loaded:")
    print(f"   🖼️  Total samples: {len(X)}")
    print(f"   👥 Unique people: {len(set(y))}")
    
    return np.array(X), np.array(y)

print("✅ Dataset loading function defined")


✅ Dataset loading function defined


## 🚀 Chạy Training Pipeline


In [7]:
# Load training dataset
print("🚀 Loading training dataset...")
start_time = time.time()

X_train, y_train = load_dataset(dataset_train, use_augmentation=True)

load_time = time.time() - start_time
print(f"⏱️  Training data loaded in {load_time:.2f} seconds")

# Load validation dataset
print("\n🚀 Loading validation dataset...")
start_time = time.time()

X_val, y_val = load_dataset(dataset_val, use_augmentation=False)

load_time = time.time() - start_time
print(f"⏱️  Validation data loaded in {load_time:.2f} seconds")


🚀 Loading training dataset...
🔄 Loading dataset from: FaceDataset/train/

👤 Processing ben_afflek...
   📸 Found 14 valid faces

👤 Processing elton_john...
   📸 Found 16 valid faces

👤 Processing mindy_kaling...
   📸 Found 22 valid faces

👤 Processing messi...
   📸 Found 15 valid faces

👤 Processing jerry_seinfeld...
   📸 Found 21 valid faces

📊 Dataset loaded:
   🖼️  Total samples: 352
   👥 Unique people: 5
⏱️  Training data loaded in 2.80 seconds

🚀 Loading validation dataset...
🔄 Loading dataset from: FaceDataset/val/

👤 Processing ben_afflek...
   📸 Found 5 valid faces

👤 Processing elton_john...
   📸 Found 5 valid faces

👤 Processing mindy_kaling...
   📸 Found 5 valid faces

👤 Processing messi...
   📸 Found 5 valid faces

👤 Processing jerry_seinfeld...
   📸 Found 5 valid faces

📊 Dataset loaded:
   🖼️  Total samples: 25
   👥 Unique people: 5
⏱️  Validation data loaded in 0.68 seconds


## 🧠 Trích xuất Face Embeddings và Train SVM


In [8]:
# Tạo embeddings cho training data
print("🧠 Creating training embeddings...")
start_time = time.time()

X_train_emb = []
y_train_emb = []

for i, face in enumerate(X_train):
    if i % 50 == 0:
        print(f"   Processing {i+1}/{len(X_train)} faces...")
    
    embedding = get_face_embedding(face)
    if embedding is not None:
        X_train_emb.append(embedding)
        y_train_emb.append(y_train[i])

X_train_emb = np.array(X_train_emb)
y_train_emb = np.array(y_train_emb)

processing_time = time.time() - start_time
print(f"⏱️  Training embeddings created in {processing_time:.2f} seconds")
print(f"📊 Valid embeddings: {len(X_train_emb)}/{len(X_train)}")

# Tạo embeddings cho validation data
print("\n🧠 Creating validation embeddings...")
start_time = time.time()

X_val_emb = []
y_val_emb = []

for i, face in enumerate(X_val):
    if i % 20 == 0:
        print(f"   Processing {i+1}/{len(X_val)} faces...")
    
    embedding = get_face_embedding(face)
    if embedding is not None:
        X_val_emb.append(embedding)
        y_val_emb.append(y_val[i])

X_val_emb = np.array(X_val_emb)
y_val_emb = np.array(y_val_emb)

processing_time = time.time() - start_time
print(f"⏱️  Validation embeddings created in {processing_time:.2f} seconds")
print(f"📊 Valid embeddings: {len(X_val_emb)}/{len(X_val)}")


🧠 Creating training embeddings...
   Processing 1/352 faces...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━

In [9]:
# Train SVM Classifier
print("🎯 Training SVM Classifier...")
print("=" * 50)

# Normalize embeddings
print("🔄 Normalizing embeddings...")
in_encoder = Normalizer(norm='l2')
X_train_norm = in_encoder.transform(X_train_emb)
X_val_norm = in_encoder.transform(X_val_emb)
print("✅ Embeddings normalized")

# Encode labels
print("🔄 Encoding labels...")
out_encoder = LabelEncoder()
out_encoder.fit(y_train_emb)
y_train_encoded = out_encoder.transform(y_train_emb)
y_val_encoded = out_encoder.transform(y_val_emb)
print("✅ Labels encoded")

print(f"\n📊 Training data:")
print(f"   Samples: {len(X_train_norm)}")
print(f"   Features: {X_train_norm.shape[1]}")
print(f"   Classes: {len(out_encoder.classes_)}")

print(f"\n📊 Validation data:")
print(f"   Samples: {len(X_val_norm)}")
print(f"   Features: {X_val_norm.shape[1]}")
print(f"   Classes: {len(set(y_val_emb))}")

# Train SVM
print("\n🚀 Training SVM...")
start_time = time.time()

svm_model = SVC(kernel='rbf', probability=True, C=1.0, gamma='scale', random_state=42)
svm_model.fit(X_train_norm, y_train_encoded)

training_time = time.time() - start_time
print(f"⏱️  SVM trained in {training_time:.2f} seconds")

# Predictions
print("🔍 Making predictions...")
y_train_pred = svm_model.predict(X_train_norm)
y_val_pred = svm_model.predict(X_val_norm)

# Calculate accuracy
train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
val_accuracy = accuracy_score(y_val_encoded, y_val_pred)

print(f"\n📊 Results:")
print(f"   Training Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"   Validation Accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")

print("\n✅ SVM training completed!")


🎯 Training SVM Classifier...
🔄 Normalizing embeddings...
✅ Embeddings normalized
🔄 Encoding labels...
✅ Labels encoded

📊 Training data:
   Samples: 352
   Features: 512
   Classes: 5

📊 Validation data:
   Samples: 25
   Features: 512
   Classes: 5

🚀 Training SVM...


⏱️  SVM trained in 0.04 seconds
🔍 Making predictions...

📊 Results:
   Training Accuracy: 1.0000 (100.00%)
   Validation Accuracy: 1.0000 (100.00%)

✅ SVM training completed!


## 💾 Lưu Model đã train


In [10]:
# Tạo thư mục output
os.makedirs("FaceDataset", exist_ok=True)

# Lưu SVM model
print("💾 Saving SVM model...")
svm_path = "FaceDataset/SVM_classifier.sav"
with open(svm_path, 'wb') as f:
    pickle.dump(svm_model, f)
print(f"✅ SVM model saved: {svm_path}")

# Lưu label encoder
print("💾 Saving label encoder...")
encoder_path = "FaceDataset/label_encoder.pkl"
with open(encoder_path, 'wb') as f:
    pickle.dump(out_encoder, f)
print(f"✅ Label encoder saved: {encoder_path}")

# Lưu normalizer
print("💾 Saving normalizer...")
normalizer_path = "FaceDataset/embedding_normalizer.pkl"
with open(normalizer_path, 'wb') as f:
    pickle.dump(in_encoder, f)
print(f"✅ Normalizer saved: {normalizer_path}")

# Lưu embeddings
print("💾 Saving embeddings...")
embeddings_npz_path = "FaceDataset/faces_dataset_embeddings.npz"
savez_compressed(embeddings_npz_path, X_train_emb, y_train_emb, X_val_emb, y_val_emb)
print(f"✅ Embeddings saved: {embeddings_npz_path}")

# Kiểm tra file sizes
svm_size = os.path.getsize(svm_path) / (1024 * 1024)
encoder_size = os.path.getsize(encoder_path) / 1024  # KB
normalizer_size = os.path.getsize(normalizer_path) / 1024  # KB
embeddings_size = os.path.getsize(embeddings_npz_path) / (1024 * 1024)

print(f"\n📊 Model file sizes:")
print(f"   SVM Classifier: {svm_size:.2f} MB")
print(f"   Label Encoder: {encoder_size:.2f} KB")
print(f"   Normalizer: {normalizer_size:.2f} KB")
print(f"   Embeddings: {embeddings_size:.2f} MB")

print(f"\n🎉 All models saved successfully!")


💾 Saving SVM model...
✅ SVM model saved: FaceDataset/SVM_classifier.sav
💾 Saving label encoder...
✅ Label encoder saved: FaceDataset/label_encoder.pkl
💾 Saving normalizer...
✅ Normalizer saved: FaceDataset/embedding_normalizer.pkl
💾 Saving embeddings...
✅ Embeddings saved: FaceDataset/faces_dataset_embeddings.npz

📊 Model file sizes:
   SVM Classifier: 0.72 MB
   Label Encoder: 0.51 KB
   Normalizer: 0.11 KB
   Embeddings: 0.68 MB

🎉 All models saved successfully!


## 📋 Tổng kết Training


In [11]:
print("🎉 FACE RECOGNITION TRAINING COMPLETED!")
print("=" * 60)

print(f"📊 Training Summary:")
print(f"   👥 People in dataset: {len(out_encoder.classes_)}")
print(f"   🖼️  Training samples: {len(X_train_emb)}")
print(f"   🖼️  Validation samples: {len(X_val_emb)}")
print(f"   🎯 Training Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")
print(f"   🎯 Validation Accuracy: {val_accuracy:.4f} ({val_accuracy*100:.2f}%)")

print(f"\n📁 Output Files:")
print(f"   🧠 SVM Classifier: FaceDataset/SVM_classifier.sav")
print(f"   🏷️  Label Encoder: FaceDataset/label_encoder.pkl")
print(f"   📏 Normalizer: FaceDataset/embedding_normalizer.pkl")
print(f"   🧠 Embeddings: FaceDataset/faces_dataset_embeddings.npz")

print(f"\n🚀 Next Steps:")
print(f"   1. Test the model: python test_models.py")
print(f"   2. Use in application: Load SVM_classifier.sav")
print(f"   3. Add new people: Retrain with new data")

print(f"\n✅ Model is ready to use!")

# Hiển thị danh sách người trong dataset
print(f"\n👥 People in dataset:")
for i, person_name in enumerate(out_encoder.classes_, 1):
    print(f"   {i:2d}. {person_name}")


🎉 FACE RECOGNITION TRAINING COMPLETED!
📊 Training Summary:
   👥 People in dataset: 5
   🖼️  Training samples: 352
   🖼️  Validation samples: 25
   🎯 Training Accuracy: 1.0000 (100.00%)
   🎯 Validation Accuracy: 1.0000 (100.00%)

📁 Output Files:
   🧠 SVM Classifier: FaceDataset/SVM_classifier.sav
   🏷️  Label Encoder: FaceDataset/label_encoder.pkl
   📏 Normalizer: FaceDataset/embedding_normalizer.pkl
   🧠 Embeddings: FaceDataset/faces_dataset_embeddings.npz

🚀 Next Steps:
   1. Test the model: python test_models.py
   2. Use in application: Load SVM_classifier.sav
   3. Add new people: Retrain with new data

✅ Model is ready to use!

👥 People in dataset:
    1. ben_afflek
    2. elton_john
    3. jerry_seinfeld
    4. messi
    5. mindy_kaling


In [1]:
import pickle
import numpy as np
from PIL import Image
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from numpy import expand_dims, asarray
import ipywidgets as widgets
from IPython.display import display, Image as IPImage
import io

# Load các model đã lưu
with open('FaceDataset/SVM_classifier.sav', 'rb') as f:
    svm_model = pickle.load(f)

with open('FaceDataset/label_encoder.pkl', 'rb') as f:
    out_encoder = pickle.load(f)

with open('FaceDataset/embedding_normalizer.pkl', 'rb') as f:
    in_encoder = pickle.load(f)

# Khởi tạo MTCNN và FaceNet (nếu chưa có)
mtcnn = MTCNN()
facenet = FaceNet()

# Hàm extract_face và get_face_embedding (copy từ cell trước nếu cần)
def extract_face(image_bytes, required_size=(160, 160)):
    """Trích xuất khuôn mặt từ bytes của ảnh sử dụng MTCNN"""
    try:
        image = Image.open(io.BytesIO(image_bytes))
        image = image.convert('RGB')
        pixels = asarray(image)
        
        results = mtcnn.detect_faces(pixels)
        
        if results:
            x1, y1, width, height = results[0]['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            
            face = pixels[y1:y2, x1:x2]
            
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array = asarray(image)
            
            return face_array
        
        return None
    
    except Exception as e:
        print(f"⚠️ Error processing image: {e}")
        return None

def get_face_embedding(face_pixels):
    """Lấy face embedding từ FaceNet"""
    try:
        samples = expand_dims(face_pixels, axis=0)
        yhat = facenet.embeddings(samples)
        return yhat[0]
    
    except Exception as e:
        print(f"⚠️ Error getting embedding: {e}")
        return None

# Hàm predict person
def predict_person(image_bytes):
    face = extract_face(image_bytes)
    if face is None:
        return 'No face detected'
    
    embedding = get_face_embedding(face)
    if embedding is None:
        return 'Error getting embedding'
    
    # Normalize embedding
    embedding_norm = in_encoder.transform(embedding.reshape(1, -1))
    
    # Predict label và probability
    pred_label = svm_model.predict(embedding_norm)
    pred_prob = svm_model.predict_proba(embedding_norm)
    
    # Decode label
    person_name = out_encoder.inverse_transform(pred_label)[0]
    confidence = max(pred_prob[0]) * 100
    
    return f'Predicted: {person_name} (Confidence: {confidence:.2f}%)'

# Widget upload file
uploader = widgets.FileUpload(accept='image/*', multiple=False)  # Chỉ chấp nhận file ảnh

# Output để hiển thị kết quả
output = widgets.Output()

def on_upload_change(change):
    with output:
        output.clear_output()
        if uploader.value:
            try:
                # Truy cập tuple và lấy file đầu tiên
                file_info = uploader.value[0]
                image_bytes = file_info['content']
                
                # Hiển thị preview ảnh
                display(IPImage(data=image_bytes, width=300))
                
                # Predict
                result = predict_person(image_bytes)
                print(result)
            except IndexError:
                print("⚠️ No file uploaded or upload failed.")
            except KeyError as e:
                print(f"⚠️ Key error in file info: {e}. Check ipywidgets version.")

uploader.observe(on_upload_change, names='value')

# Hiển thị widget
display(uploader, output)

FileUpload(value=(), accept='image/*', description='Upload')

Output()